In [47]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#Load Training Dataset
trainFeatures = pd.read_csv("features.csv",header=None)
trainLabels = pd.read_csv("labels_reg.csv", header=None)

In [48]:
print(trainFeatures.shape, trainLabels.shape)

(121275, 109) (121275, 1)


In [70]:
#Load Full Testing Dataset

flights = pd.read_csv("jan2016.csv")
testFlights = flights.loc[flights["ORIGIN"]=="BOS"]

print(len(testFlights))

testFlights.to_csv("jan2016boston.csv")


8791


In [12]:
# Extract Test Features
# Feature Vector - 109 features (36 # and 73 categories)
# * MONTH * SCHED DEPART TIME * CARRIER * DEST AIRPORT *

testFeatures = np.zeros((len(testFlights), 109))
carriers = ["B6", "AA", "DL", "UA", "WN", "NK", "EV", "VX", "AS", "OO"]
airports = ['DCA', 'ORD', 'ATL', 'LGA', 'BWI', 'PHL', 'EWR', 'JFK', 'LAX', 'SFO', 'MCO', 'CLT', 'DTW', 'DFW', 'DEN', 'FLL', 'MIA', 'IAD', 'CLE', 'RSW', 'MSP', 'SEA', 'RIC', 'TPA', 'PBI', 'RDU', 'BNA', 'PHX', 'PIT', 'MDW', 'IAH', 'BUF', 'LAS', 'SLC', 'SJU', 'SAN', 'STL', 'HOU', 'AUS', 'JAX', 'MSY', 'CMH', 'MKE', 'PDX', 'MCI', 'IND', 'CHS', 'CVG', 'SAV', 'DAL', 'MYR', 'LGB', 'SJC', 'STT', 'SRQ', 'ACK', 'ACY', 'OAK', 'MVY', 'SMF', 'ORF', 'SDF', 'MSN']

for i in range(0, len(testFlights)):
    line = testFlights.iloc[i]
    month = line.loc["MONTH"]
    departTime = int(line.loc["DEP_BLOCK"])
   
    carrier = line.loc["CARRIER"]
    carrierInd = carriers.index(carrier)
    
    destAirport = line.loc["DEST"]
    destInd = airports.index(destAirport)
    
    testFeatures[i,month-1]=1
    testFeatures[i, departTime+12] = 1
    testFeatures[i, carrierInd+12+24-1] =1
    testFeatures[i, destInd+12+24+10-1] =1

In [49]:
# Extract Test Labels (Regression)
testLabels = np.zeros(len(testFlights))
one, two, three, four = 0, 0, 0, 0

for i in range(0, len(testFlights)):
    line = testFlights.iloc[i]
    delay = line.loc["DEP_DELAY"]
    testLabels[i] = delay
        
print(testLabels)

[  23.  203.   53. ...,    0.    0.    8.]


In [17]:
# Extract Test Labels (Class)

testLabels = np.zeros(len(testFlights))
one, two, three, four = 0, 0, 0, 0

for i in range(0, len(testFlights)):
    line = testFlights.iloc[i]
    delay = line.loc["DEP_DELAY"]
    
    if (delay <= 30):
        one+=1
        testLabels[i] = 0
        
    elif (30 < delay <= 60):
        two+=1
        testLabels[i] = 1
        
    elif (60 < delay <= 120):
        three+=1
        testLabels[i] = 2
        
    elif (delay > 120):
        four+=1
        testLabels[i] = 3

print(one,two,three, four)
print(testLabels)

7998 356 288 149
[ 0.  3.  1. ...,  0.  0.  0.]


In [50]:
trainLabels = np.ravel(trainLabels)
testLabels = np.ravel(testLabels)

print(trainFeatures.shape, trainLabels.shape, testFeatures.shape, testLabels.shape)

(121275, 109) (121275,) (8791, 109) (8791,)


In [80]:
print("Test Flights")
print("\nSan Francisco")
print(testFlights.iloc[1].loc["CARRIER"], testFlights.iloc[1].loc["DEST"], testFlights.iloc[1].loc["DEP_DELAY"])
print("\nDenver")
print(testFlights.iloc[136].loc["CARRIER"], testFlights.iloc[136].loc["DEST"], testFlights.iloc[136].loc["DEP_DELAY"])
print("\nMiami")
print(testFlights.iloc[1256].loc["CARRIER"], testFlights.iloc[1256].loc["DEST"], testFlights.iloc[1256].loc["DEP_DELAY"])
print("\nNew York")
print(testFlights.iloc[4064].loc["CARRIER"], testFlights.iloc[4064].loc["DEST"], testFlights.iloc[4064].loc["DEP_DELAY"])

Test Flights

San Francisco
VX SFO 203

Denver
WN DEN 23

Miami
AA MIA 65

New York
B6 JFK 0


In [63]:
# Regression
from sklearn.ensemble import RandomForestRegressor

rgr = RandomForestRegressor()
rgr.fit(trainFeatures, trainLabels)
print(np.mean(testLabels - rgr.predict(testFeatures)))

-2.87962782818


In [133]:
print("Test Flights")
print("\nHouston")
ind = 250
print(testFlights.iloc[ind].loc["CARRIER"], testFlights.iloc[ind].loc["DEST"], testFlights.iloc[ind].loc["DAY"], testFlights.iloc[ind].loc["DEP_TIME"], testFlights.iloc[ind].loc["CRS_ARR_TIME"])
print("Prediction: ",rgr.predict(testFeatures[ind].reshape(1, -1)), "Actual: ",testLabels[ind])
print("\nSan Francisco")
ind=282
print(testFlights.iloc[ind].loc["CARRIER"], testFlights.iloc[ind].loc["DEST"], testFlights.iloc[ind].loc["DAY"], testFlights.iloc[ind].loc["DEP_TIME"], testFlights.iloc[ind].loc["CRS_ARR_TIME"])
print("Prediction: ",rgr.predict(testFeatures[ind].reshape(1, -1)), "Actual: ",testLabels[ind])
print("\nPalm Beach")
ind=6000
print(testFlights.iloc[ind].loc["CARRIER"], testFlights.iloc[ind].loc["DEST"], testFlights.iloc[ind].loc["DAY"], testFlights.iloc[ind].loc["DEP_TIME"], testFlights.iloc[ind].loc["CRS_ARR_TIME"])
print("Prediction: ",rgr.predict(testFeatures[ind].reshape(1, -1)), "Actual: ",testLabels[ind])
print("\nNew York")
ind=7411
print(testFlights.iloc[ind].loc["CARRIER"], testFlights.iloc[ind].loc["DEST"], testFlights.iloc[ind].loc["DAY"], testFlights.iloc[ind].loc["DEP_TIME"], testFlights.iloc[ind].loc["CRS_ARR_TIME"])
print("Prediction: ",rgr.predict(testFeatures[ind].reshape(1, -1)), "Actual: ",testLabels[ind])

Test Flights

Houston
WN HOU 9 5:50 935
Prediction:  [ 0.1375] Actual:  0.0

San Francisco
UA SFO 17 14:10 1753
Prediction:  [ 78.4] Actual:  125.0

Palm Beach
B6 PBI 25 8:45 1212
Prediction:  [ 17.09721758] Actual:  18.0

New York
DL LGA 27 20:00 2125
Prediction:  [ 33.76] Actual:  57.0


In [84]:
print(np.mean(rgr.predict(testFeatures)))
print(np.max(rgr.predict(testFeatures)))
print(np.min(rgr.predict(testFeatures)))
print(np.median(rgr.predict(testFeatures)))

13.3683094344
314.716666667
0.0
8.60006504989
